In [ ]:
# --- ADIM 0: GEREKLİ KÜTÜPHANELERİ YÜKLEME ---
# Bu hücreyi çalıştırdıktan sonra mutlaka "Runtime" -> "Restart runtime" yapın!

# Önce mevcut kurulumları temizle
!pip uninstall -y bitsandbytes

# Bitsandbytes'ı en son sürümle yükle
!pip install -U bitsandbytes

# Unsloth ve diğer bağımlılıkları yükle
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate

print("Kütüphaneler yüklendi. Şimdi 'Runtime' -> 'Restart runtime' yapın ve sonraki hücreyi çalıştırın.")


Found existing installation: bitsandbytes 0.46.1
Uninstalling bitsandbytes-0.46.1:
  Successfully uninstalled bitsandbytes-0.46.1
  Using cached bitsandbytes-0.46.1-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
Using cached bitsandbytes-0.46.1-py3-none-manylinux_2_24_x86_64.whl (72.9 MB)
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-8qrierym/unsloth_754e939d296740b3b9e408e6870f079d
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-8qrierym/unsloth_754e939d296740b3b9e408e6870f079d
  Resolved https://github.com/unslothai/unsloth.git to commit 7dd4e1dd5d9fceeeb08612e0716f041bee8144ee
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached xformers-0.0.25.post1-cp311-cp311-manylinux2014_x86_64.whl.metadata (1.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.6/222.6 MB 6.4 MB/s eta 0:00:00
Kütüphanele

In [ ]:
# --- ADIM 0: GEREKLİ KÜTÜPHANELERİ YÜKLEME ---
# Bu hücreyi çalıştırdıktan sonra mutlaka "Runtime" -> "Restart runtime" yapın!

#
# =============================================================================
# RUNTIME RESTART SONRASI ÇALIŞTIRIN - YENİ HÜCRE
# =============================================================================

# Gerekli kütüphaneleri import et
import os
import torch
from datasets import Dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import FastLanguageModel
import json
import re

# Bitsandbytes kurulumunu kontrol et
try:
    import bitsandbytes as bnb
    print(f"✓ Bitsandbytes sürümü: {bnb.__version__}")
except ImportError:
    print("❌ Bitsandbytes kurulu değil! Lütfen runtime'ı restart edin.")

# --- 1. ADIM: KONFİGÜRASYON ---

# Model ayarları
MODEL_NAME = "unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit"
DATASET_FILE_PATH = "/content/test_qa.json"
OUTPUT_ADAPTER_DIR = "dora-finetuned-r1-oracle"

# Hiperparametreler
MAX_SEQ_LENGTH = 2048
DORA_RANK = 16
DORA_ALPHA = 32

# --- 2. ADIM: VERİ SETİNİ YÜKLEME ---

def load_data_from_custom_format(filepath):
    """
    JSON dosyasını okur ve question/answer formatını input/output formatına dönüştürür.
    """
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            content = f.read()
    except FileNotFoundError:
        print(f"HATA: Veri seti dosyası bulunamadı: {filepath}")
        return []

    data = []
    success_count = 0
    error_count = 0

    # Önce doğrudan JSON array formatını dene
    try:
        parsed_data = json.loads(content)
        if isinstance(parsed_data, list):
            for item in parsed_data:
                # question/answer -> input/output dönüşümü
                if 'question' in item and 'answer' in item:
                    data.append({
                        'input': item['question'],
                        'output': item['answer']
                    })
                    success_count += 1
                else:
                    error_count += 1
        else:
            error_count += 1
    except json.JSONDecodeError:
        # JSON array formatı değilse, tek tek satırları dene (JSONL formatı)
        print("JSON array formatı değil, JSONL formatı deneniyor...")
        for line_num, line in enumerate(content.splitlines(), 1):
            line = line.strip()
            if not line:
                continue
            try:
                parsed = json.loads(line)
                # question/answer -> input/output dönüşümü
                if 'question' in parsed and 'answer' in parsed:
                    data.append({
                        'input': parsed['question'],
                        'output': parsed['answer']
                    })
                    success_count += 1
                else:
                    error_count += 1
                    if error_count <= 5:
                        print(f"Satır {line_num}: 'question' ve 'answer' alanları bulunamadı")
            except json.JSONDecodeError as e:
                error_count += 1
                if error_count <= 5:
                    print(f"Satır {line_num} ayrıştırılamadı: {str(e)[:100]}...")

        # Eğer JSONL da çalışmadıysa, ```json blokları dene
        if not data:
            print("JSONL formatı da çalışmadı, ```json blokları deneniyor...")
            json_blocks = re.findall(r'```json\s*\n(.*?)\n```', content, re.DOTALL)

            for i, block in enumerate(json_blocks, 1):
                try:
                    cleaned_block = block.strip()
                    parsed = json.loads(cleaned_block)
                    # question/answer -> input/output dönüşümü
                    if 'question' in parsed and 'answer' in parsed:
                        data.append({
                            'input': parsed['question'],
                            'output': parsed['answer']
                        })
                        success_count += 1
                    else:
                        error_count += 1
                        if error_count <= 5:
                            print(f"Blok {i}: 'question' ve 'answer' alanları bulunamadı")
                except json.JSONDecodeError as e:
                    error_count += 1
                    if error_count <= 5:
                        print(f"Blok {i} ayrıştırılamadı: {str(e)[:100]}...")

    print(f"\n✓ Başarılı: {success_count} örnek")
    print(f"❌ Hatalı: {error_count} örnek")

    if not data:
        raise ValueError("HATA: Hiçbir geçerli örnek yüklenemedi!")

    return data

# Veri setini yükle
print("Veri seti yükleniyor...")
json_data = load_data_from_custom_format(DATASET_FILE_PATH)
dataset = Dataset.from_list(json_data)

print(f"\n📊 Veri seti özeti:")
print(f"Toplam örnek sayısı: {len(dataset)}")
print(f"İlk örnek: {dataset[0]}")

# --- 3. ADIM: MODELİ YÜKLEME ---

print(f"\n🤖 Model yükleniyor: {MODEL_NAME}")

# GPU bellek durumunu kontrol et
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"Mevcut VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

try:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=MODEL_NAME,
        max_seq_length=MAX_SEQ_LENGTH,
        dtype=None,
        load_in_4bit=True,
    )
    print("✓ Model başarıyla yüklendi!")
except Exception as e:
    print(f"❌ Model yükleme hatası: {e}")
    print("Alternatif model denenecek...")

    # Yedek plan: Farklı bir DeepSeek model
    try:
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name="unsloth/llama-3.2-3b-instruct-bnb-4bit",  # Alternatif model
            max_seq_length=MAX_SEQ_LENGTH,
            dtype=None,
            load_in_4bit=True,
        )
        print("✓ Alternatif model başarıyla yüklendi!")
    except Exception as e2:
        print(f"❌ Alternatif model de yüklenemedi: {e2}")
        raise

# --- 4. ADIM: DORA ADAPTÖRÜ ---

print("\n🔧 DoRA adaptörleri ekleniyor...")

model = FastLanguageModel.get_peft_model(
    model,
    r=DORA_RANK,
    lora_alpha=DORA_ALPHA,
    lora_dropout=0.1,
    bias="none",
    use_gradient_checkpointing="unsloth",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                   "gate_proj", "up_proj", "down_proj"],
    use_dora=True,
)

print("✓ DoRA adaptörleri eklendi!")
model.print_trainable_parameters()

# --- 5. ADIM: VERİYİ FORMATLA ---

def formatting_prompts_func(examples):
    """
    Oracle-specific formatting function.
    input/output alanlarını kullanarak chat formatına dönüştürür.
    """
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []

    for input_text, output_text in zip(inputs, outputs):
        # Oracle SCM için özel sistem mesajı ekle
        messages = [
            {"role": "system", "content": "You are an Oracle Fusion Cloud SCM expert assistant. Provide accurate and helpful information about Oracle manufacturing, supply chain management, and related processes."},
            {"role": "user", "content": input_text},
            {"role": "assistant", "content": output_text},
        ]

        try:
            formatted = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=False
            )
            texts.append(formatted)
        except Exception as e:
            # Eğer chat template desteklenmiyorsa, basit format kullan
            print(f"Chat template hatası, basit format kullanılıyor: {e}")
            simple_format = f"<|system|>You are an Oracle Fusion Cloud SCM expert assistant. Provide accurate and helpful information about Oracle manufacturing, supply chain management, and related processes.<|end|>\n<|user|>{input_text}<|end|>\n<|assistant|>{output_text}<|end|>"
            texts.append(simple_format)

    return {"text": texts}

print("\n📝 Veri seti formatlanıyor...")
dataset = dataset.map(formatting_prompts_func, batched=True)
print("✓ Veri seti formatlandı!")

# İlk örneki kontrol et
print(f"\nFormatlanmış örnek:\n{dataset[0]['text'][:500]}...")

# --- 6. ADIM: EĞİTİM ---

print("\n🚀 Eğitim başlatılıyor...")

# Training arguments - Oracle domain için optimize edilmiş
training_args = TrainingArguments(
    per_device_train_batch_size=1,  # Bellek için düşürüldü
    gradient_accumulation_steps=8,  # Effective batch size: 8
    warmup_steps=10,
    num_train_epochs=2,  # 1648 örnek için optimize edildi
    learning_rate=1e-4,  # Biraz daha konservatif
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="cosine",  # Cosine scheduler daha iyi sonuç verebilir
    seed=42,
    output_dir="outputs",
    report_to="none",
    save_steps=100,
    save_total_limit=3,
    dataloader_pin_memory=False,
    group_by_length=True,  # Benzer uzunluktaki örnekleri grupla
)

# SFTTrainer'ı oluştur
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,  # Tokenizer'ı geri ekledik
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    args=training_args,
    packing=False,
    dataset_num_proc=2,
)

# Eğitimi başlat
print("=" * 50)
print("EĞİTİM BAŞLIYOR...")
print("=" * 50)

trainer_stats = trainer.train()

print("=" * 50)
print("EĞİTİM TAMAMLANDI!")
print("=" * 50)
print(f"Eğitim istatistikleri: {trainer_stats}")

# --- 7. ADIM: MODELİ KAYDET ---

print(f"\n💾 Model kaydediliyor: {OUTPUT_ADAPTER_DIR}")

model.save_pretrained(OUTPUT_ADAPTER_DIR)
tokenizer.save_pretrained(OUTPUT_ADAPTER_DIR)

print(f"✅ İşlem tamamlandı!")
print(f"📁 Adaptörler '{OUTPUT_ADAPTER_DIR}' klasörüne kaydedildi.")

# --- 8. ADIM: TEST (İsteğe bağlı) ---

print("\n🧪 Hızlı test:")

# Fast inference için modeli hazırla
FastLanguageModel.for_inference(model)

# Test soruları - veri setinden örnekler
test_questions = [
    "What is the purpose of Oracle Fusion Cloud SCM Using Manufacturing?",
    "How do you implement resource allocation in Oracle Fusion Cloud SCM?",
    "What are the key features of Oracle Fusion Cloud SCM in relation to supply chain optimization?"
]

for test_question in test_questions[:2]:  # İlk 2 soruyu test et
    test_messages = [
        {"role": "system", "content": "You are an Oracle Fusion Cloud SCM expert assistant."},
        {"role": "user", "content": test_question},
    ]

    try:
        test_prompt = tokenizer.apply_chat_template(
            test_messages,
            tokenize=False,
            add_generation_prompt=True
        )
    except:
        # Basit format kullan
        test_prompt = f"<|system|>You are an Oracle Fusion Cloud SCM expert assistant.<|end|>\n<|user|>{test_question}<|end|>\n<|assistant|>"

    inputs = tokenizer([test_prompt], return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            use_cache=True,
            temperature=0.3,  # Daha deterministik cevaplar için
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"\n🔍 Soru: {test_question}")

    # Cevabı temizle
    assistant_marker = 'assistant'
    if assistant_marker in response:
        clean_response = response.split(assistant_marker)[-1].strip()
        if clean_response.startswith('>'):
            clean_response = clean_response[1:].strip()
    else:
        clean_response = response.split(test_question)[-1].strip()

    print(f"🤖 Cevap: {clean_response}")
    print("-" * 80)

print("\n🎉 Tüm işlemler başarıyla tamamlandı!")
print(f"📈 Toplam {len(dataset)} örnek ile {training_args.num_train_epochs} epoch eğitim tamamlandı.")
print(f"💾 Model '{OUTPUT_ADAPTER_DIR}' klasörüne kaydedildi.")

✓ Bitsandbytes sürümü: 0.46.1
Veri seti yükleniyor...

✓ Başarılı: 1648 örnek
❌ Hatalı: 0 örnek

📊 Veri seti özeti:
Toplam örnek sayısı: 1648
İlk örnek: {'input': 'What is the purpose of Oracle Fusion Cloud SCM Using Manufacturing?', 'output': "The document 'Oracle Fusion Cloud SCM Using Manufacturing' provides detailed guidance on leveraging Oracle Fusion Cloud SCM for efficient manufacturing processes. It covers topics like material management, production planning, and inventory control."}

🤖 Model yükleniyor: unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit
GPU: Tesla T4
Mevcut VRAM: 15.8 GB
==((====))==  Unsloth 2025.8.4: Fast Llama patching. Transformers: 4.55.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is en

<string>:37: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.


✓ Model başarıyla yüklendi!

🔧 DoRA adaptörleri ekleniyor...
✓ DoRA adaptörleri eklendi!
trainable params: 43,319,296 || all params: 8,073,580,544 || trainable%: 0.5366

📝 Veri seti formatlanıyor...


Map:   0%|          | 0/1648 [00:00<?, ? examples/s]

✓ Veri seti formatlandı!

Formatlanmış örnek:
<｜begin▁of▁sentence｜>You are an Oracle Fusion Cloud SCM expert assistant. Provide accurate and helpful information about Oracle manufacturing, supply chain management, and related processes.<｜User｜>What is the purpose of Oracle Fusion Cloud SCM Using Manufacturing?<｜Assistant｜>The document 'Oracle Fusion Cloud SCM Using Manufacturing' provides detailed guidance on leveraging Oracle Fusion Cloud SCM for efficient manufacturing processes. It covers topics like material management, production plann...

🚀 Eğitim başlatılıyor...


Unsloth: Tokenizing ["text"]:   0%|          | 0/1648 [00:00<?, ? examples/s]

EĞİTİM BAŞLIYOR...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,648 | Num Epochs = 2 | Total steps = 412
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 43,319,296 of 8,073,580,544 (0.54% trained)


Step,Training Loss
1,3.960400
2,4.267200
3,4.061700
4,4.383600
5,4.127100
6,3.733700
7,3.550500
8,3.434800
9,3.105000
10,2.932800


In [ ]:
# Oracle Model Merge ve GGUF Çevirme
import torch, gc
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig
import os
import shutil

print("🚀 Oracle Model Merge İşlemi Başlıyor...")

# Bellek temizliği
torch.cuda.empty_cache()
gc.collect()

# Sizin model yolunuz
ADAPTER_PATH = "/content/dora-finetuned-gemma-oracle"
OUTPUT_PATH = "/content/merged_oracle_model"

try:
    # DoRA config'i oku
    print("📖 Adapter konfigürasyonu okunuyor...")
    config = PeftConfig.from_pretrained(ADAPTER_PATH)
    print(f"✅ Base model: {config.base_model_name_or_path}")

    # Base modeli CPU'da yükle (bellek tasarrufu için)
    print("🔄 Base model yükleniyor...")
    base_model = AutoModelForCausalLM.from_pretrained(
        config.base_model_name_or_path,
        torch_dtype=torch.float16,
        device_map={"": "cpu"},
        low_cpu_mem_usage=True,
        trust_remote_code=True
    )

    # Tokenizer yükle
    print("🔤 Tokenizer yükleniyor...")
    tokenizer = AutoTokenizer.from_pretrained(
        config.base_model_name_or_path,
        trust_remote_code=True
    )

    # Pad token ekle (Gemma için gerekli)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # Adapter'ı yükle ve merge et
    print("🔗 Adapter yükleniyor ve merge ediliyor...")
    model = PeftModel.from_pretrained(
        base_model,
        ADAPTER_PATH,
        device_map={"": "cpu"}
    )

    print("⚡ Merge işlemi yapılıyor...")
    merged_model = model.merge_and_unload()

    # Merged model'i kaydet
    print(f"💾 Merged model kaydediliyor: {OUTPUT_PATH}")
    merged_model.save_pretrained(OUTPUT_PATH, safe_serialization=True)
    tokenizer.save_pretrained(OUTPUT_PATH)

    print("✅ Merge işlemi tamamlandı!")

    # Bellek temizliği
    del base_model, model, merged_model
    torch.cuda.empty_cache()
    gc.collect()

except Exception as e:
    print(f"❌ Merge hatası: {e}")
    print("Alternatif yöntem deneniyor...")

# ===== GGUF ÇEVİRME BÖLÜMÜ =====
print("\n🔄 GGUF formatına çevirme başlıyor...")

try:
    # Unsloth ile GGUF çevirme
    from unsloth import FastLanguageModel

    # Merged model'i Unsloth ile yükle
    print("📂 Merged model Unsloth ile yükleniyor...")
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=OUTPUT_PATH,
        max_seq_length=2048,
        dtype=None,
        load_in_4bit=False,  # Zaten merged model
    )

    # GGUF'a çevir
    print("🔄 GGUF formatına çevriliyor...")
    model.save_pretrained_gguf(
        "oracle_final_gguf",
        tokenizer,
        quantization_method="q4_k_m"
    )

    print("✅ GGUF çevirme başarılı!")

except Exception as e:
    print(f"❌ Unsloth GGUF çevirme hatası: {e}")
    print("🔧 Manuel GGUF çevirme deneniyor...")

    try:
        # Llama.cpp ile manuel çevirme
        print("📥 Llama.cpp indiriliyor...")
        !git clone https://github.com/ggerganov/llama.cpp.git

        print("🔨 Llama.cpp derleniyor...")
        !cd llama.cpp && make

        print("🔄 FP16 GGUF'a çevriliyor...")
        !python llama.cpp/convert.py {OUTPUT_PATH} --outtype f16 --outfile oracle_model_f16.gguf

        print("📦 Q4_K_M quantization yapılıyor...")
        !./llama.cpp/quantize oracle_model_f16.gguf oracle_model_q4_k_m.gguf q4_k_m

        print("✅ Manuel GGUF çevirme başarılı!")

        # Dosyaları organize et
        os.makedirs("oracle_final_gguf", exist_ok=True)
        shutil.move("oracle_model_q4_k_m.gguf", "oracle_final_gguf/oracle_model.q4_k_m.gguf")

    except Exception as e2:
        print(f"❌ Manuel GGUF çevirme de başarısız: {e2}")

# ===== DOSYALARI ZİPLE VE İNDİR =====
print("\n📦 Dosyalar paketleniyor...")

try:
    # Merged HF model'i zipla
    if os.path.exists(OUTPUT_PATH):
        shutil.make_archive("oracle_merged_hf", 'zip', OUTPUT_PATH)
        print("✅ HuggingFace modeli paketlendi: oracle_merged_hf.zip")

    # GGUF dosyalarını zipla
    if os.path.exists("oracle_final_gguf"):
        shutil.make_archive("oracle_gguf_model", 'zip', "oracle_final_gguf")
        print("✅ GGUF modeli paketlendi: oracle_gguf_model.zip")

    # İndir
    from google.colab import files

    print("\n⬇️ Dosyalar indiriliyor...")
    if os.path.exists("oracle_merged_hf.zip"):
        files.download("oracle_merged_hf.zip")

    if os.path.exists("oracle_gguf_model.zip"):
        files.download("oracle_gguf_model.zip")

    print("🎉 Tüm işlemler tamamlandı!")

except Exception as e:
    print(f"❌ Paketleme hatası: {e}")

# ===== ÖZETİ YAZDIR =====
print("\n" + "="*50)
print("📋 İŞLEM ÖZETİ")
print("="*50)
print(f"✅ Adapter yolu: {ADAPTER_PATH}")
print(f"✅ Merged model: {OUTPUT_PATH}")
print("✅ İndirilen dosyalar:")
print("   - oracle_merged_hf.zip (HuggingFace formatı)")
print("   - oracle_gguf_model.zip (Ollama için GGUF)")
print("="*50)

In [ ]:
# Oracle Model - SADECE .gguf DOSYASI İNDİRME
import torch, gc
import os
from google.colab import files

print("🚀 GGUF Dosyası Çevirme - Direkt İndirme!")

# Bellek temizliği
torch.cuda.empty_cache()
gc.collect()

# Model yolu
ADAPTER_PATH = "/content/dora-finetuned-r1-oracle"

# Model kontrolü
if not os.path.exists(ADAPTER_PATH):
    print("❌ Model bulunamadı!")
    print("Mevcut:", os.listdir('/content/'))
    exit()

print(f"✅ Model bulundu: {ADAPTER_PATH}")

try:
    print("📂 Model yükleniyor...")
    from unsloth import FastLanguageModel

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=ADAPTER_PATH,
        max_seq_length=2048,
        dtype=None,
        load_in_4bit=False,
    )

    print("⚡ Fast inference mode...")
    FastLanguageModel.for_inference(model)

    print("🔄 GGUF Q4_K_M formatına çevriliyor...")

    # GGUF çevir
    model.save_pretrained_gguf(
        "oracle_gguf_temp",
        tokenizer,
        quantization_method="q4_k_m"
    )

    print("✅ GGUF çevirme başarılı!")

    # Model'i bellekten temizle
    del model, tokenizer
    torch.cuda.empty_cache()
    gc.collect()

    # GGUF klasöründeki dosyaları listele
    gguf_files = os.listdir("oracle_gguf_temp")
    print(f"📁 Oluşturulan dosyalar: {gguf_files}")

    # .gguf dosyasını bul
    gguf_file = None
    for file in gguf_files:
        if file.endswith('.gguf'):
            gguf_file = file
            break

    if gguf_file:
        gguf_path = os.path.join("oracle_gguf_temp", gguf_file)

        # Dosya boyutunu göster
        file_size = os.path.getsize(gguf_path) / 1024 / 1024
        print(f"📄 GGUF dosyası: {gguf_file} ({file_size:.1f} MB)")

        # Dosyayı ana dizine taşı ve yeniden adlandır
        final_name = "oracle-scm-q4_k_m.gguf"
        os.rename(gguf_path, final_name)

        print(f"📄 Yeniden adlandırıldı: {final_name}")

        # Direkt .gguf dosyasını indir
        print("⬇️ GGUF dosyası indiriliyor...")
        files.download(final_name)

        print("🎉 BAŞARILI! .gguf dosyası indirildi!")

    else:
        print("❌ .gguf dosyası bulunamadı!")
        print("Mevcut dosyalar:", gguf_files)

except Exception as e:
    print(f"❌ Hata: {e}")
    print("\n🔄 Alternatif yöntem deneniyor...")

    # Yedek plan: Adapter dosyalarından en önemli olanları indir
    try:
        adapter_files = os.listdir(ADAPTER_PATH)
        important_files = [f for f in adapter_files if f.endswith(('.safetensors', '.bin', '.json'))]

        print(f"📁 Adapter dosyaları: {important_files}")

        for file in important_files[:3]:  # İlk 3 önemli dosya
            file_path = os.path.join(ADAPTER_PATH, file)
            print(f"⬇️ {file} indiriliyor...")

            # Dosyayı kopyala ve indir
            import shutil
            shutil.copy(file_path, f"oracle_adapter_{file}")
            files.download(f"oracle_adapter_{file}")

    except Exception as e2:
        print(f"❌ Yedek plan da başarısız: {e2}")

# Özet
print("\n" + "="*50)
print("📋 ÖZET")
print("="*50)
print("✅ İndirilen: oracle-scm-q4_k_m.gguf")
print("🚀 Ollama kullanımı:")
print("   1. Modelfile oluştur:")
print("      FROM ./oracle-scm-q4_k_m.gguf")
print("   2. ollama create oracle-scm -f Modelfile")
print("   3. ollama run oracle-scm")
print("="*50)